In [26]:
!pip install nltk
import pandas as pd
import nltk
import re

# Reading Data

In [27]:
messages = pd.read_csv("../input/smsspamcollection/SMSSpamCollection",sep="\t",names=["label","message"])
messages.head()

In [28]:
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.stem import PorterStemmer
#from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Preprocessing

In [29]:
#Stemming
ps = PorterStemmer()
#wordnet = WordNetLemmatizer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub("[^a-zA-Z]"," ",messages["message"][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [31]:
tf.__version__

In [32]:
voc_size = 10000

In [33]:
X = messages["message"]
Y = messages["label"]

In [34]:
encoded_output=pd.get_dummies(Y)
encoded_output

In [35]:
encoded_output = encoded_output["spam"]

In [36]:
#One hot encoding text data
encoded_corpus = [one_hot(word,voc_size)for word in corpus]
encoded_corpus

In [37]:
#finding max length of the sentence with max length from the corpus
max=len(encoded_corpus[i])
for i in range(0,len(encoded_corpus)-1):
    if(len(encoded_corpus[i+1])>=max):
        max=len(encoded_corpus[i+1])
    else:
        max=len(encoded_corpus[i])
print(max)

In [38]:
sent_length = max

In [39]:
#Padding
padded_corpus = pad_sequences(encoded_corpus,padding="pre",maxlen=13)

In [40]:
padded_corpus

# Creating Model

In [41]:
vector_features = 30
model = Sequential()
model.add(Embedding(voc_size,vector_features,input_length=sent_length))
model.add(LSTM(50))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

# Model Fitting

In [42]:
import numpy as np
X = np.array(padded_corpus)
Y = np.array(encoded_output)

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [44]:
len(X_train)

In [45]:
y_test

In [46]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=40)

In [47]:
y_pred = model.predict_classes(X_test)

In [48]:
y_pred

# Checking Accuracy

In [49]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print("Accuracy score : \n",accuracy_score(y_test,y_pred))
print("Confusion matrix : \n",confusion_matrix(y_test,y_pred))
print("Classification report : \n",classification_report(y_test,y_pred))